In [12]:
# System imports
import os
import sys
import functools as ft

# 3rd-party imports
import numpy as np
import keras
from ipyparallel.datapub import publish_data
import ipyparallel as ipp

# Local imports
from mnist import load_data
from updatingplot import UpdatingPlot
from mlextras import build_and_train
import paramspan as ps
#from paramspan import ParamSpanWidget

In [13]:
# Set up engines
c = ipp.Client()
lv = c.load_balanced_view()
dv = c.direct_view()
print(c.ids)

[0, 1]


In [14]:
# Make sure tensorflow modules are available remotely
def set_path(path):
    global sys
    import sys
    sys.path = path
dv.apply(set_path, sys.path)

<AsyncResult: set_path>

In [4]:
x_train, y_train, x_test, y_test = load_data()

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
y_train shape: (60000, 10)
y_test shape: (10000, 10)


In [5]:
# Training config
# Hold these parameters constant
batch_size = 128
n_epochs = 16
valid_frac = 0.17
checkpoint_dir = '/global/cscratch1/sd/oevans/cori-interactive-dl/mnist-hpo'
checkpoint_file = os.path.join(checkpoint_dir, 'model_single.h5')
n_samples = 1000

const = dict(
    checkpoint_file=checkpoint_file, verbose=0,
    x_train=x_train[:n_samples], 
    y_train=y_train[:n_samples],
    valid_frac=valid_frac,
    batch_size=batch_size, n_epochs=n_epochs,
)

In [6]:
run_training = ft.partial(
    build_and_train,
    **const
)

plot_metrics = ft.partial(
    UpdatingPlot, 
    y=['loss', 'acc'],
    xlim=[0, n_epochs],
    xlabel='epochs',
    ylabel='training metrics'
)

# Single Parameter Set

In [21]:
h1 = 16
h2 = 16
h3 = 32
dropout=0.5
optimizer = 'Adadelta'

n_samples = 1000

opts = dict(
    h1=h1, h2=h2, h3=h3,
    dropout=dropout, 
    optimizer=optimizer
)

# Several Parameter Sets

In [22]:
n_hpo_trials = 4
grid_h1 = np.random.choice([4, 8, 16, 32, 64], size=n_hpo_trials)
grid_h2 = np.random.choice([4, 8, 16, 32, 64], size=n_hpo_trials)
grid_h3 = np.random.choice([8, 16, 32, 64, 128], size=n_hpo_trials)
grid_dropout = np.random.rand(n_hpo_trials)
grid_optimizer = np.random.choice(['Adadelta', 'Adam', 'Nadam'], size=n_hpo_trials)

In [23]:
psw = ps.ParamSpanWidget(run_training, plot_metrics)

psw.set_params(
    h1=grid_h1,
    h2=grid_h2,
    h3=grid_h3,
    dropout=grid_dropout,
    optimizer=grid_optimizer
)

psw.submit_computations()

psw

ParamSpanWidget(children=(Output(layout=Layout(border='1px solid', height='500px', overflow_x='scroll', overfl…